# ADD ELEMENTS TO THE ONTOLOGY OF EUCLID'S BOOK 1

In [ ]:
from __future__ import annotations

import pathlib
import sys

NOTEBOOK_DIR = pathlib.Path.cwd()
if '_NB_SYS_PATH_ADJUSTED' not in globals():
    sys.path.insert(0, str(NOTEBOOK_DIR))
    _NB_SYS_PATH_ADJUSTED = True

import pandas as pd
import rdflib

from modules import legacy_utils

URIREF = rdflib.URIRef
RDF_TYPE = rdflib.RDF.type
RDFS_LABEL = rdflib.RDFS.label
SKOS_PREFLABEL = rdflib.SKOS.prefLabel
CONCEPT_CLASS = URIREF("https://www.foom.com/core#concept")
NAMESPACE = "<https://www.foom.com/core>"

In [ ]:
from modules import rdf_utils

def load_graph(input_ttl: pathlib.Path) -> rdflib.Graph:
    graph = rdf_utils.read_graph(str(input_ttl))
    return graph

#  read input ontology file
INPUT_TTL = NOTEBOOK_DIR.parent / "ontologies" / "ontology_euclid_book1.ttl"
g = load_graph(INPUT_TTL)


In [ ]:
# add contains_concepts relations to postulates and common notions
from modules import legacy_utils

INPUT_FILE = NOTEBOOK_DIR / "input" / "addenda_contains_concept.csv"
OUTPUT_FOLDER = NOTEBOOK_DIR / "ontologies"

def add_concept(
    g: rdflib.Graph,
    concept_string: str
):
    concept_label = concept_string.replace("_", " ")
    concept_iri = legacy_utils.create_iri(f"Concept: {concept_string}")
    g.add((URIREF(concept_iri), RDF_TYPE, CONCEPT_CLASS))
    g.add((URIREF(concept_iri), RDFS_LABEL, rdflib.Literal(f"Concept: {concept_label}")))
    g.add((URIREF(concept_iri), SKOS_PREFLABEL, rdflib.Literal(concept_label)))
    return g, concept_iri

def contains_concepts(
    g: rdflib.Graph,
    input_file: pathlib.Path = INPUT_FILE
):
    # read csv file
    df = pd.read_csv(input_file)
    concepts = set()
    for _, row in df.iterrows():
        subject_iri = URIREF(row['subject'])
        objects = {concept.strip() for concept in row['objects'].split(',')}
        for object_string in objects:
            object_iri = legacy_utils.create_iri(f"Concept: {object_string}")
            concepts.add(object_string)
            # add triples to the graph
            g.add((
                URIREF(subject_iri),
                URIREF("https://www.foom.com/core#contains_concept"),
                URIREF(object_iri)))
            g.add((
                URIREF(object_iri),
                URIREF("https://www.foom.com/core#is_concept_in"),
                URIREF(subject_iri)))
    return g, concepts


def write_csv(
        iris_df: pd.DataFrame, 
        *, 
        filename: str = "concepts.csv", 
        output_dir: pathlib.Path = OUTPUT_FOLDER
    ) -> pathlib.Path:
    output_dir.mkdir(parents=True, exist_ok=True)
    output_path = output_dir / filename
    iris_df.to_csv(output_path, index=False)
    print(f"Saved iris to {output_path}")
    return output_path

def add_concepts_and_contains_concept(
    g: rdflib.Graph,
    input_file: pathlib.Path = INPUT_FILE
):
    g, concepts = contains_concepts(g, input_file)
    print(f"Added {len(concepts)} new concepts to the graph.")
    iris = set()
    for concept_string in concepts:
        g, concept_iri = add_concept(g, concept_string)
        iris.add(concept_iri)
    iris_df = pd.DataFrame({"iri": sorted(iris)})
    write_csv(iris_df)
    return g

g = add_concepts_and_contains_concept(g, INPUT_FILE)